qgrid - An interactive grid for viewing and editing pandas DataFrames
=======================================================
Qgrid is an Jupyter notebook widget which uses a javascript library called **SlickGrid** to render pandas DataFrames within a Jupyter notebook.


## Overview
* [SlickGrid](https://github.com/mleibman/SlickGrid) is a javascript grid which allows users to scroll, sort, 
and filter hundreds of thousands of rows with extreme responsiveness.  
* [Pandas](https://github.com/pydata/pandas) is a powerful data analysis / manipulation library for Python, and DataFrames are the primary way of storing and manipulating two-dimensional data in pandas.

[Qgrid](https://github.com/quantopian/qgrid) renders pandas DataFrames as SlickGrids, which enables users to explore the entire contents of a DataFrame using intuitive sorting and filtering controls.  It's built on the ipywidget framework and is designed to be used in Jupyter notebook, Jupyterhub, or Jupyterlab

## What's new

#### Column options and new "live-updating" API methods - as of 1.1.0
* Column options can be provided via the `show_grid` method. Options can be provided for all columns via the `column_options` parameter, and for individual columns via the `column_definitions` parameter.
* Added `edit_cell`, `change_selection`, `toggle_editable` methods for updating the state of an existing grid widget without having to call `show_grid`.
* Updated the `add_row` method so that the caller can specify the values for the new row via the `row` parameter. This will allow people to add rows to a qgrid instance even if it's showing a DataFrame that doesn't have an integer index.
* Updated the `remove_row` method so that the indices of the rows to remove can optionally be provided via the `rows` parameter.
* Fixed issue where moving the scroll bar around a bunch of times quickly can cause a series of grid refreshes to occur.

#### Multi-index support - as of 1.0.6-beta.6
* Improves support for viewing DataFrames with a MultiIndex.
* Cells are merged vertically (similar to how pandas does it) to make it easier to identify the levels of the index.
* Sorting or grouping any column other than level 0 of the multi-index results in the DataFrame returning to it's       normal behavior of never merging cells vertically.
* Column header is hidden for unnamed levels of the index (instead of showing "level_0", "level_1", etc)


## Example 1 - Render a DataFrame with many different types of columns

In [ ]:
import numpy as np
import pandas as pd
import qgrid

In [12]:
randn = np.random.randn
df_types = pd.DataFrame({
    'A' : pd.Series(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09'],index=list(range(9)),dtype='datetime64[ns]'),
    'B' : pd.Series(randn(9),index=list(range(9)),dtype='float32'),
    'C' : pd.Categorical(["washington", "adams", "washington", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'D' : ["foo", "bar", "buzz", "bippity","boppity", "foo", "foo", "bar", "zoo"] })
df_types['E'] = df_types['D'] == 'foo'

In [13]:
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

If you make any sorting/filtering changes, or edit the grid by double clicking, you can retrieve a copy of your DataFrame which reflects these changes by calling `get_changed_df` on the `QgridWidget` instance returned by `show_grid`.

In [2]:
qgrid_widget.get_changed_df()

,A,B,C,D,E
0,2013-01-01,-0.295272,washington,foo,True
1,2013-01-02,0.577026,adams,bar,False
2,2013-01-03,-0.648877,washington,buzz,False
3,2013-01-04,0.807211,madison,bippity,False
4,2013-01-05,0.305350,lincoln,boppity,False
5,2013-01-06,0.496477,jefferson,foo,True
6,2013-01-07,1.141522,hamilton,foo,True
7,2013-01-08,-0.580613,roosevelt,bar,False
8,2013-01-09,-0.558293,kennedy,zoo,False


## Example 2 - Render a DataFrame with 1 million rows


In [3]:
# set the default max number of rows to 10 so the larger DataFrame we render don't take up to much space 
qgrid.set_grid_option('maxVisibleRows', 10)

df_scale = pd.DataFrame(np.random.randn(1000000, 4), columns=list('ABCD'))

# duplicate column B as a string column, to test scalability for text column filters
df_scale['B (as str)'] = df_scale['B'].map(lambda x: str(x))
q_scale = qgrid.show_grid(df_scale, show_toolbar=True, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})
q_scale

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [4]:
q_scale.get_changed_df()

,A,B,C,D,B (as str)
0,0.622497,0.755190,0.497520,-0.315271,0.7551897430304813
1,1.024678,-1.214926,0.187070,0.327590,-1.2149258275853978
2,1.274869,-0.319840,-1.667119,0.976567,-0.3198396893799533
3,-2.324339,-0.895539,-0.348008,0.591475,-0.8955392826107696
4,1.323460,-0.117315,1.326693,0.328479,-0.11731524756595418
5,-0.391053,-0.077133,-1.749508,-0.640557,-0.07713343184023945
6,-0.277586,-1.364218,-1.277528,-0.070731,-1.3642182894429062
7,0.537181,-0.111977,-0.634691,-1.481313,-0.11197718168284267
8,1.423914,-0.699852,1.597679,0.597839,-0.6998519394706131
9,0.306547,0.441822,1.174826,0.701492,0.4418217313629605
